<a href="https://colab.research.google.com/github/hyeonsang010716/Recommendation-system/blob/main/neural_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#gpu 사용하기
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-05-07 05:30:57--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  1.54MB/s    in 0.6s    

2024-05-07 05:30:58 (1.54 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
users = []
items = []
ratings = []
with open("ml-latest-small/ratings.csv" , "r") as f:
  f.readline()
  for line in f:
    uid , mid , rating , _ = line.split(",")
    users.append(int(uid))
    items.append(int(mid))
    ratings.append(float(rating))

  users = torch.tensor(users)
  items = torch.tensor(items)
  ratings = torch.tensor(ratings)
  print(ratings)

tensor([4., 4., 4.,  ..., 5., 5., 3.])


In [ ]:
n_nnz = len(users)
train_size = int(n_nnz * 0.8)
shuffled_indices = torch.randperm(n_nnz)
#그냥 저장하면 main memory에 저장되기 때문에 gpu를 사용하기 위해서 gpu 상에 올려줘야한다.
users_train = users[shuffled_indices[:train_size]].to(device)
items_train = items[shuffled_indices[:train_size]].to(device)
ratings_train = ratings[shuffled_indices[:train_size]].to(device)

users_test = users[shuffled_indices[train_size:]].to(device)
items_test = items[shuffled_indices[train_size:]].to(device)
ratings_test = ratings[shuffled_indices[train_size:]].to(device)


In [ ]:
import torch.nn as nn
#neural collaborative filtering
# user vector 와 item vector를 concat 해서 mult layer n
n_users = max(users) + 1
n_items = max(items) + 1
n_factors = 10
dropout_rate = 0.4
P = torch.randn(n_users , n_factors , requires_grad = True , device = device)
Q = torch.randn(n_items , n_factors , requires_grad = True , device = device)
model = nn.Sequential(
    nn.Linear(n_factors*2 , 10),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(10,5),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(5,1)
).to(device)

In [ ]:
mse = nn.MSELoss()
optim = torch.optim.Adam(list(model.parameters()) + [P,Q], lr = 0.1)
for epoch in range(3000):
  model.train()
  x = torch.cat((P[users_train] , Q[items_train] ), dim = 1)
  h = model(x).squeeze()
  cost = mse(h , ratings_train)
  optim.zero_grad()
  cost.backward()
  optim.step()

  with torch.no_grad():
    if epoch % 100 == 0:
      model.eval()
      x_train = torch.cat((P[users_train] , Q[items_train] ), dim = 1)
      h_train = model(x_train).squeeze()
      train_mse = mse(h_train , ratings_train)

      x_test = torch.cat((P[users_test] , Q[items_test] ), dim = 1)
      h_test = model(x_test).squeeze()
      test_mse = mse(h_test , ratings_test)
      print(f"epoch : {epoch} , train_mse: {train_mse.item()} , test_mse : {test_mse.item()}")



epoch : 0 , train_mse: 1.294806957244873 , test_mse : 1.3332263231277466
epoch : 100 , train_mse: 0.7022123336791992 , test_mse : 0.834744930267334
epoch : 200 , train_mse: 0.6876100301742554 , test_mse : 0.8356665372848511
epoch : 300 , train_mse: 0.6808602809906006 , test_mse : 0.8362711071968079
epoch : 400 , train_mse: 0.6733590960502625 , test_mse : 0.8358569145202637
epoch : 500 , train_mse: 0.6734362840652466 , test_mse : 0.8389440774917603
epoch : 600 , train_mse: 0.6725870370864868 , test_mse : 0.8370033502578735
epoch : 700 , train_mse: 0.6759024262428284 , test_mse : 0.8371350169181824
epoch : 800 , train_mse: 0.670719563961029 , test_mse : 0.836831271648407
epoch : 900 , train_mse: 0.673116147518158 , test_mse : 0.838914692401886
epoch : 1000 , train_mse: 0.6692038774490356 , test_mse : 0.8357536792755127
epoch : 1100 , train_mse: 0.6737407445907593 , test_mse : 0.8412884473800659
epoch : 1200 , train_mse: 0.6636024713516235 , test_mse : 0.837451159954071
epoch : 1300 , tra

In [ ]:
#
rating_matrix_train = torch.zeros(n_users , n_items)
for uid , mid , rating in zip(users_train , items_train , ratings_train):
  rating_matrix_train[uid , mid] = rating
rating_matrix_train = rating_matrix_train.to(device)

rating_matrix_test = torch.zeros(n_users , n_items)
for uid , mid , rating in zip(users_test , items_test , ratings_test):
  rating_matrix_test[uid , mid] = rating
rating_matrix_test = rating_matrix_test.to(device)

In [ ]:
n_factors = 30
model = nn.Sequential(
    nn.Linear(n_users , n_factors),
    nn.Sigmoid(),
    nn.Linear(n_factors , n_users)
).to(device)

In [ ]:
mse = nn.MSELoss()
optim = torch.optim.Adam(model.parameters() , lr = 0.1 , weight_decay = 0.001)
nnz_train = rating_matrix_train != 0
nnz_test = rating_matrix_test != 0
for epoch in range(30):
  h = model(rating_matrix_train.T)
  cost = mse(h.T[nnz_train] , rating_matrix_train[nnz_train])
  optim.zero_grad()
  cost.backward()
  optim.step()
  with torch.no_grad():
    h_train = model(rating_matrix_train.T)
    train_mse = mse(h_train.T[nnz_train] , rating_matrix_train[nnz_train])

    h_test = model(rating_matrix_test.T)
    test_mse = mse(h_test.T[nnz_test] , rating_matrix_test[nnz_test])
    print(f"epoch : {epoch} , train_mse : {train_mse.item()} , test_mse : {test_mse.item()}")

epoch : 0 , train_mse : 6.065615177154541 , test_mse : 5.773454189300537
epoch : 1 , train_mse : 1.6659064292907715 , test_mse : 1.6142293214797974
epoch : 2 , train_mse : 1.7187237739562988 , test_mse : 1.704734444618225
epoch : 3 , train_mse : 2.878044366836548 , test_mse : 2.969487190246582
epoch : 4 , train_mse : 2.9270591735839844 , test_mse : 3.0922937393188477
epoch : 5 , train_mse : 2.361156940460205 , test_mse : 2.465439558029175
epoch : 6 , train_mse : 1.7006028890609741 , test_mse : 1.7586747407913208
epoch : 7 , train_mse : 1.2922803163528442 , test_mse : 1.3429473638534546
epoch : 8 , train_mse : 1.2224193811416626 , test_mse : 1.2848525047302246
epoch : 9 , train_mse : 1.3712282180786133 , test_mse : 1.4530433416366577
epoch : 10 , train_mse : 1.5603132247924805 , test_mse : 1.6544336080551147
epoch : 11 , train_mse : 1.6455957889556885 , test_mse : 1.7417880296707153
epoch : 12 , train_mse : 1.5861457586288452 , test_mse : 1.6726640462875366
epoch : 13 , train_mse : 1.43